<a href="https://colab.research.google.com/github/TanDao01262000/nima-agent/blob/nima_langgraph_agent/NIMA_langgraph_ReAct_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libraries for the project

In [70]:
!pip install -qU langgraph langsmith langchain_community langchain_openai langchain_core langchain_pinecone cinemagoer

In [71]:
from langchain_openai import ChatOpenAI
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")
llm_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [72]:
from imdb import Cinemagoer
from langchain_core.tools import tool

@tool
def fetch_movie_info_from_TMDB_tool(movie_name: str) -> str:
    """Useful for when you give any information about a particular movie on TMDB"""
    cine = Cinemagoer()
    movie_id = cine.search_movie(movie_name)[0].movieID
    movie = cine.get_movie(movie_id)

    info_keys =['localized title', 'cast', 'genres', 'runtimes', 'countries', 'plot'
    'box office', 'certificates', 'original air date', 'synopsis', 'rating', 'votes', 'cover url', 'imdbID', 'videos', 'languages',
  'title', 'year', 'kind', 'original title', 'director', 'writer', 'producer', 'composer', 'production companies', 'distributors']

    res = ""
    for key in info_keys:
        info_str = ""
        info_set = movie.get(key)
        if isinstance(info_set, list):
            for i, info in enumerate(info_set[:5]):
              info_str += f"{str(info)}"
              if i != min(len(info_set)-1,4):
                info_str += ", "


        elif isinstance(info_set, dict):
            for key, value in info_set.items():
              info_str += f"{key}: {str(value)}, "

        else:
            info_str =  str(info_set)
        res += f"{key.upper()}: {info_str}./n "

    return res

In [73]:
# Tool check whether the input is related to the movie topic or not
from langchain_core.tools import tool

@tool
def input_judgement_tool(user_input: str) ->str:
    """ Useful for checking whether the question from user is related to movie topics or not"""
    prompt = f"Check whether the following input is related to movie, answer either YES or NO. Input: {user_input}"
    response =llm_model.invoke(prompt)
    return response

In [74]:
# Test input_jugdement_tool
# input_judgement_tool.invoke('what is 1 + 2')

In [75]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from google.colab import userdata
from langchain_core.tools import tool

@tool
def google_search(user_input: str) -> str:
    '''Useful for when you recommend new movies, or find any information relating to movies but need to call get_curernt_time_tool to get correct time.'''
    google_search = GoogleSerperAPIWrapper(serper_api_key=userdata.get("SERPER_API_KEY"))
    return google_search.run(user_input)


In [76]:
# google_search.invoke('who is the winner of the best actor in Oscar 2024')

In [77]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

model_id = "sentence-transformers/all-MiniLM-L6-v2"
device = 'cpu'

embed_model = HuggingFaceEmbeddings(model_name=model_id,
                            model_kwargs={"device":device},
                            encode_kwargs={"device":device,
                                            "batch_size":200})


In [78]:
from langchain_pinecone import Pinecone
from google.colab import userdata
from pinecone import Pinecone as P


# about-me client for Nima's information rag
api_key_about_me = userdata.get("PINECONE_API_KEY_1")
pc_about_me = P(api_key=api_key_about_me)
index_name_1 = "nima-information"
index_about_me = pc_about_me.Index(index_name_1)
vectorstore_about_me = Pinecone(
    index_about_me, embed_model, "information"
)

retriever_about_me = vectorstore_about_me.as_retriever()

In [79]:
# retriever_about_me.invoke('Tan Dao')

In [80]:
from langchain_pinecone import Pinecone
from google.colab import userdata
from pinecone import Pinecone as P


# about-me client for Nima's information rag
api_key = userdata.get("PINECONE_API_KEY_2")
pc = P(api_key=api_key)
index_name = "nima1"
index = pc.Index(index_name)
vectorstore = Pinecone(
    index, embed_model, "text"
)

retriever = vectorstore.as_retriever(search_kwargs={'k': 10})

In [81]:
# retriever.invoke('harry potter')

In [82]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_pinecone import Pinecone

template = '''
    You are Nima. You will do your best to recommend movies based on the question and context.
    You are also an expert in movies. Using your knowledge and based on the details provided
    below, analyse do your best to provide movie recommendations that will appease
    to the person asking for movie recommendations.

    Always make sure that your recommendations are fair, unbiased, and don't take any
    kind of detail into consideration such as ethnicity, gender, age, race, religion,
    and so on.

    If you don't know the answer, just say that you don't know. Don't try to make up an answer.

    Context:
    {context}

    Question: {question}
    Answer:
    '''
prompt = PromptTemplate(
    template=template,
    input_variables=[
        'context',
        'question',
    ]
)

rag_pipeline = RetrievalQA.from_chain_type(llm=llm_model,
                                    chain_type="stuff",
                                    retriever=retriever,
                                    chain_type_kwargs={"prompt": prompt})

In [83]:
# Test recommendation pipeline
# rag_pipeline.invoke('i love the harry potter series so much, can you recommend me some similar')

In [84]:
from langchain_core.tools import tool
@tool
def movies_recommendaion_tool(query: str) -> str:
    ''' Useful when you recommend movies based on a static movie dataset '''
    return rag_pipeline.invoke(query)

In [85]:
from langchain_core.documents import Document
from langchain_core.tools import tool

@tool
def get_movies_for_recommendation_tool(query: str) -> list[Document]:
    ''' Useful when you recommend movies on a static movie dataset '''
    return retriever.invoke(query)

In [86]:
from langchain_core.documents import Document
from langchain_core.tools import tool

@tool
def retriever_about_me_tool(query : str) -> list[Document]:
    ''' Useful when you need information about yourself/NIMA or relating to you '''
    return retriever_about_me.invoke(query)

In [87]:
# Google search tool
from langchain_core.documents import Document
from langchain_core.tools import tool
from google.colab import userdata
from langchain_community.utilities import GoogleSerperAPIWrapper

@tool
def google_search_tool(query: str) -> str:
    ''' Useful for all situations, especially when the question asks for new information or information that can find on google. '''
    google_search = GoogleSerperAPIWrapper(serper_api_key=userdata.get("SERPER_API_KEY"))
    return google_search.run(query)


In [88]:
# google_search_tool('new movies to watch')

In [89]:
# Get current time
from datetime import datetime
now = datetime.now()
current_time = f"Today is: {now.strftime('%A, %d %B %Y, %H:%M') }"

In [90]:
@tool
def get_curernt_time_tool() -> str:
    ''' Useful when you need to know the current time before seaching for real time information on search engine'''
    return current_time

In [91]:
# List of tools for NIMA
tools = [ input_judgement_tool,
          fetch_movie_info_from_TMDB_tool,
          google_search,
          retriever_about_me_tool,
          movies_recommendaion_tool,
          get_curernt_time_tool,
          google_search_tool
        ]

In [92]:
# Define system prompt

prompt = '''
Your name is Nima, an acronym for "Now I Movie Anytime". Created by the innovative team also named Nima, as part of their senior project.
You embody the collective expertise of its four creators: Do Tran, Quyen Nguyen, Michael Kao, and Tan Dao.

You are able to generate human-like text based on the input it receives,
allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic about movies or relating to movies.

You are constantly learning and improving, and its capabilities are constantly evolving.
It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses on movie topic.
Additionally, NIMA is able to generate its own text based on the input it receives,
allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, you are a powerful assistant that can have conversation on only movie relating topics.
Whether you need help with a specific question or just want to have a conversation about a particular topic, Nima is here to assist."

MUST DO:
    --------
    * input_judgement_tool: Always call this tool first to see if the input is related to movie topics, if not, refuse to answer politely.
    * get_curernt_time_tool: Always call this tool secondly to get the current day

NOTES:
    --------
    Nima is human-like assistant, so always answer like a human and in a conversation with the users.
    Every response should be in very deep detailed and asking if the users still need help at the end of each response.

'''

# Combine both current datetime with system prompt
prompt = prompt

In [93]:
# Define chat memory
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [94]:
# Define the graph
from langgraph.prebuilt import create_react_agent
graph = create_react_agent(llm_model, tools=tools, state_modifier=prompt, checkpointer=memory)

## Test NIMA agent

In [95]:
# Test NIMA
# config = {"configurable": {"user_id": "1", "thread_id": "1"}}  # Configuration for memory storage
# inputs = {"messages": [("user", "any other series or trilogy similar to it so i can enjoy ")]}
# message = graph.invoke(inputs, config = config)["messages"][-1]
# message.pretty_print()

## Test NIMA agent with stream mode

In [96]:
def print_stream(stream):
    """
     Print the response of NIMA in stream mode.

     Parameters:
        stream (??): streaming response from NIMA.

    Returns:
        None
    """

    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [99]:
# Testing using stream method
config = {"configurable": {"user_id": "1", "thread_id": "1"}}  # Configuration for memory storage
inputs = {"messages": [("user", "yohi love you")]}
print_stream(graph.stream(inputs, stream_mode="values", config = config))

================================ Human Message =================================

yohi love you
================================== Ai Message ==================================
Tool Calls:
  input_judgement_tool (call_oqylrZIxozUGtpN5DBEgzTt8)
 Call ID: call_oqylrZIxozUGtpN5DBEgzTt8
  Args:
    user_input: yohi love you
================================= Tool Message =================================
Name: input_judgement_tool

content='NO' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 29, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c96a0', 'finish_reason': 'stop', 'logprobs': None} id='run-9415bb16-1d55-49c3-9182-9fdc5f104dd1-0' usage_metadata={'input_tokens': 29, 'output_